In [ ]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np

print(f"Working directory: {os.getcwd()}")
# experiment_path = "experiments\\2024-11-04_15-34-38\\9fba3d9_dr-sac_algorithm_environment_zero-init\\sac_l2f_false"
experiment_path = "experiments\\2024-11-04_16-29-38\\9fba3d9_dr-sac_algorithm_environment_zero-init\\sac_l2f_false"
print(f"Loading returns from {experiment_path}")

returns = []
for seed in sorted(os.listdir(experiment_path)):
    return_path_set = os.path.join(experiment_path, seed, "return.json.set")
    return_path = os.path.join(experiment_path, seed, "return.json")
    if os.path.exists(return_path_set):
        with open(return_path, "r") as f:
            returns.append(json.load(f))
print(f"Loaded {len(returns)} returns")

In [ ]:
plt.hist([r["parameters"]["dynamics"]["mass"] for r in returns])
plt.ylabel("Count")
plt.xlabel("Mass [kg]")
plt.show()

In [ ]:
thrusts = np.array([np.dot(r["parameters"]["dynamics"]["rotors"][0]["thrust_curve"], [1, 1, 1]) for r in returns])
weights = np.array([r["parameters"]["dynamics"]["mass"] * 9.81 for r in returns])
thrust_to_weight = thrusts * 4 / weights

plt.hist(thrust_to_weight)
plt.ylabel("Count")
plt.xlabel("Thrust to Weight Ratio")
plt.show()

In [ ]:
inertia_xx = np.array([r["parameters"]["dynamics"]["J"][0][0] for r in returns])
plt.hist(inertia_xx)
plt.ylabel("Count")
plt.xlabel("Inertia (xx) [kg*m^2]")
plt.show()

In [ ]:
rotor_distances = np.array([np.linalg.norm(r["parameters"]["dynamics"]["rotors"][0]["pose"]["position"]) for r in returns])

plt.hist(rotor_distances * 2)
plt.ylabel("Count")
plt.xlabel("Diameter [m]")
plt.show()

In [ ]:
torques = rotor_distances * np.sqrt(2) * thrusts * 9.81
torque_to_inertia = torques / inertia_xx
plt.hist(torque_to_inertia)
plt.ylabel("Count")
plt.xlabel("Torque to Inertia Ratio")
plt.show()

In [ ]:
plt.hist(thrust_to_weight / torque_to_inertia)
plt.ylabel("Count")
plt.xlabel("Thrust to Weight to Torque to Inertia Ratio")
plt.show()

In [ ]:
max_episode_lengths = [max([max(step["episode_length"]) for step in r["evaluation"]]) for r in returns]

plt.hist(max_episode_lengths)
plt.ylabel("Count")
plt.xlabel("Max Episode Lengths [steps]")
plt.show()

mean_episode_lengths = [np.mean([np.mean(step["episode_length"]) for step in r["evaluation"]]) for r in returns]

plt.hist(mean_episode_lengths)
plt.ylabel("Count")
plt.xlabel("Mean Episode Lengths [steps]")
plt.show()

final_mean_episode_length = [np.mean(r["evaluation"][-1]["episode_length"]) for r in returns]
plt.hist(final_mean_episode_length)
plt.ylabel("Count")
plt.xlabel("Final Mean Lengths [steps]")
plt.show()

steps = [step["step"] for step in returns[0]["evaluation"]]

learning_curves = [[np.mean(step["episode_length"]) for step in r["evaluation"]] for r in returns]

for el in learning_curves:
    plt.plot(steps, el)
plt.ylabel("Count")
plt.xlabel("Max Episode Lengths [steps]")
plt.show()

learning_curves_aggregate = np.stack(learning_curves, axis=0)
plt.fill_between(steps, np.mean(learning_curves_aggregate, axis=0) - np.std(learning_curves_aggregate, axis=0), np.mean(learning_curves_aggregate, axis=0) + np.std(learning_curves_aggregate, axis=0), alpha=0.5)
plt.plot(steps, np.mean(learning_curves_aggregate, axis=0))
plt.ylabel("Episode Length")
plt.xlabel("Step")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = {
    'mass': [r["parameters"]["dynamics"]["mass"] for r in returns],
    'rotor_distance': rotor_distances,
    'inertia_xx': inertia_xx,
    'thrust_to_weight': thrust_to_weight,
    'torque_to_inertia': torque_to_inertia,
    'thrust_to_weight_to_torque_to_inertia': thrust_to_weight / torque_to_inertia,
    'max_episode_length': max_episode_lengths,
    'mean_episode_length': mean_episode_lengths,
    'final_mean_episode_length': final_mean_episode_length
}
df = pd.DataFrame(data)

corr_matrix = df.corr()

fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.matshow(corr_matrix, cmap='coolwarm')

fig.colorbar(cax)

ax.set_xticks(np.arange(len(corr_matrix.columns)))
ax.set_yticks(np.arange(len(corr_matrix.columns)))
ax.set_xticklabels(corr_matrix.columns, rotation=45, ha='left')
ax.set_yticklabels(corr_matrix.columns)

for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}', ha='center', va='center', color='black')

plt.show()